In [43]:
import pandas as pd
import random

In [44]:
df = pd.read_csv('simData48Clean.csv')
df.head()

,date,event,fighter1,fighter1 - open,fighter1 - 48h,fighter1 - close,fighter2,fighter2 - open,fighter2 - 48h,fighter2 - close,winner
0,2025-03-09,UFC,Curtis Blaydes,2025-01-18 02:57:00 - price: -275,2025-03-06 15:01:00 - price: -339,2025-03-08 15:01:00 - price: -335,Rizvan Kuniev,2025-01-18 02:57:00 - price: +210,2025-03-06 15:01:00 - price: +259,2025-03-08 15:01:00 - price: +261,NaN
1,2025-03-09,UFC,Brunno Ferreira,2025-02-24 18:07:00 - price: +105,2025-03-06 21:01:00 - price: +116,2025-03-08 21:01:00 - price: +112,Armen Petrosyan,2025-02-24 18:07:00 - price: -125,2025-03-06 21:01:00 - price: -141,2025-03-08 21:01:00 - price: -134,Brunno Ferreira
2,2025-03-09,UFC,Rafael Fiziev,2025-02-27 05:11:00 - price: -139,2025-03-07 01:01:00 - price: -163,2025-03-08 23:01:00 - price: -159,Justin Gaethje,2025-02-27 05:11:00 - price: +112,2025-03-07 01:01:00 - price: +133,2025-03-08 23:01:00 - price: +131,Justin Gaethje
3,2025-03-09,UFC,Francis Marshall,2025-02-24 18:15:00 - price: +290,2025-03-06 17:01:00 - price: +247,2025-03-08 19:01:00 - price: +197,Mairon Santos,2025-02-24 18:15:00 - price: -370,2025-03-06 17:01:00 - price: -322,2025-03-08 19:01:00 - price: -245,Mairon Santos
4,2025-03-09,UFC,John Castaneda,2025-03-03 09:01:00 - price: -112,2025-03-06 15:01:00 - price: -106,2025-03-08 15:01:00 - price: -108,Chris Gutierrez,2025-03-03 09:01:00 - price: -112,2025-03-06 15:01:00 - price: -116,2025-03-08 15:01:00 - price: -112,NaN


In [45]:
def calculate_profit(wager, price):
    if price > 0:
        profit = (price/100)*wager
    else:
        profit = (100/abs(price))*wager
    return profit

In [ ]:
wager = 100
bankroll = wager * len(df)

profit_dict = {
    'favorite_at_open': bankroll,
    'favorite_at_48': bankroll,
    'favorite_at_close': bankroll,
    

    'underdog_at_open': bankroll, 
    'underdog_at_48': bankroll,
    'underdog_at_close': bankroll
}

for index, row in df.iterrows():
    fighter1 = row['fighter1']
    fighter2 = row['fighter2']
    winner = row['winner']

    #get open line
    open1 = int(row['fighter1 - open'].split('price: ')[1])
    open2 = int(row['fighter2 - open'].split('price: ')[1])

    #get 48h lines
    forty81 = int(row['fighter1 - 48h'].split('price: ')[1])
    forty82 = int(row['fighter2 - 48h'].split('price: ')[1])

    #get closing lines
    close1 = int(row['fighter1 - close'].split('price: ')[1])
    close2 = int(row['fighter2 - close'].split('price: ')[1])

    lines = {
        fighter1: [open1, forty81, close1],
        fighter2: [open2, forty82, close2]
    }


    #find who is favorite and dog at open
    if open1<open2:
        fav_at_open = fighter1
        dog_at_open = fighter2
    elif open2<open1:
        fav_at_open = fighter2
        dog_at_open = fighter1
    else:
        #if lines are dead even randomly pick a fighter
        fav_at_open = random.choice([fighter1, fighter2])
        if fav_at_open == fighter1:
            dog_at_open = fighter2
        else:
            dog_at_open = fighter1

    if fav_at_open == winner:
        profit_dict['favorite_at_open']+=calculate_profit(wager, lines[winner][0])
        profit_dict['underdog_at_open']-=wager
    elif dog_at_open == winner:
        profit_dict['underdog_at_open']+=calculate_profit(wager, lines[winner][0])
        profit_dict['favorite_at_open']-=wager



    #get favorite at 48 and dog at 48
    if forty81<forty82:
        fav_at_48 = fighter1
        dog_at_48 = fighter2
    elif forty82<forty81:
        fav_at_48 = fighter2
        dog_at_48 = fighter1
    else:
        fav_at_48 = random.choice([fighter1, fighter2])
        if fav_at_48 == fighter1:
            dog_at_open = fighter2
        else:
            dog_at_open = fighter1
    
    if fav_at_48 == winner:
        profit_dict['favorite_at_48']+=calculate_profit(wager, lines[winner][1])
        profit_dict['underdog_at_48']-=wager
    elif dog_at_48 == winner:
        profit_dict['underdog_at_48']+=calculate_profit(wager, lines[winner][1])
        profit_dict['favorite_at_48']-=wager

    if close1<close2:
        fav_at_close=fighter1
        dog_at_close = fighter2
    elif close2<close1:
        fav_at_close=fighter2
        dog_at_close = fighter1
    else:
        fav_at_close = random.choice([fighter1, fighter2])
        if fav_at_close == fighter1:
            dog_at_close = fighter2
        else:
            dog_at_close = fighter1

    if fav_at_close == winner:
        profit_dict['favorite_at_close']+=calculate_profit(wager, lines[winner][2])
        profit_dict['underdog_at_close']-=wager
    elif dog_at_close == winner:
        profit_dict['underdog_at_close']+=calculate_profit(wager, lines[winner][2])
        profit_dict['favorite_at_close']-=wager


    


for k in profit_dict:
        profit_dict[k] = (profit_dict[k] - bankroll) / bankroll

print(profit_dict)            


{'favorite_at_open': -0.032583448355837166, 'favorite_at_48': -0.021163772751808647, 'favorite_at_close': -0.021971562888926186, 'underdog_at_open': -0.10639249255285554, 'underdog_at_48': -0.09185970857035576, 'underdog_at_close': -0.08068509859691352}
